<a href="https://colab.research.google.com/github/VeraMendes/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [3]:
import re
import string
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [18]:
r.encoding = r.apparent_encoding


In [24]:
from tensorflow import keras
from tensorflow.keras.utils import get_file

shakespeare_url = "https://www.gutenberg.org/files/100/100-0.txt" 
filepath = get_file("shakespeare.txt", shakespeare_url)
with open(filepath, encoding="utf8") as f:
    shakespeare_text = f.read()

5783552/5777367 [==============================] - 3s 0us/step


In [25]:

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts([shakespeare_text])

In [26]:
import numpy as np


chars = list(set(tokenizer.word_index))

char_indices = {c:i for i,c in enumerate(chars)}
indices_char = {i:c for i,c in enumerate(chars)}

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) -1

In [27]:

# Create the Sequence Data
maxlen = 100
step = 5

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence
for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
print('sequences:', len(sequences))

sequences: 1114611


In [28]:

# Specify x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        y[i, next_chars[i]] = 1


In [29]:
x.shape

(1114611, 100, 107)

In [30]:
y.shape

(1114611, 107)

In [32]:
# build the model

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [33]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [34]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(shakespeare_text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = shakespeare_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [35]:
model.fit(x[:200000], y[:200000],
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Train on 200000 samples
Epoch 1/5
199936/200000 [============================>.] - ETA: 0s - loss: 2.6806
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "gain turn back and fly.
  RICHARD. Ay, now methinks I hear great Warwick speak.
    Ne'er may he liv"
gain turn back and fly.
  RICHARD. Ay, now methinks I hear great Warwick speak.
    Ne'er may he liv6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ld7u“ ﻿6“ld7(“B6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“u﻿6(“mWI“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“﻿6(6“ ﻿6“ld7u“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6(6“&d7u“mWI“u﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6(“ ﻿6“ ﻿6u“HW“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“u﻿mWI“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“ ﻿6“
----- diversity: 0.5
----- Generating with seed: "gain turn back and fly.
  RICHARD. Ay, now methinks I hear great Warwick speak.
    Ne'er may he liv"
gain turn back and fly.
  RICHARD. Ay, now methinks 

    But spoke the harm that is by other[T﻿6“ "63“ÆWH/(dGm3[iHI“HW“u6m [Æd7(#[“fp!L	C“&tp!		#“çH"“I&66I“ ﻿6“﻿m(R﻿"\﻿“dWI[b&6WJmlP[T﻿6“R﻿HO\6IRèu6Gu]“HWI6“mWI“ ﻿m “ ﻿d7“I6(d7[MH/ ﻿duèu“ud(z[[ip}VsL#“V&“﻿Hu è “(B__m(6“B6u “Bd(m “m“Bm(6#“,6z[CHJu“﻿HBG“m(“\﻿6B“lmJ63“dmu (7èu“BHB3[““““V7(“ÆdB6u“d_GW“&﻿d(\63“u(6\﻿è“R﻿6“ld&P“IH ﻿Bd( u#[““}7G3“6(dl“mduu“B7 HW“R6lB#[[LÆVsLV#“&6mO6W“mu“(7Od('uH/W3“tmOHW“dR“JHu“R76“pWd ﻿6W“ ﻿6“JHWu;“M﻿6B66W3“dB“ud
200000/200000 [==============================] - 55376s 277ms/sample - loss: 2.0668
Epoch 4/5
199936/200000 [============================>.] - ETA: 0s - loss: 1.9829
----- Generating text after Epoch: 3
----- diversity: 0.2
----- Generating with seed: "ight fame,
    Before young Talbot from old Talbot fly,
    The coward horse that bears me fall and "
ight fame,
    Before young Talbot from old Talbot fly,
    The coward horse that bears me fall and ﻿“ ﻿6“/6m(6“ ﻿6“_m( HWR“ ﻿6“_m(6“ ﻿6“_(mHWR6(“ ﻿m “﻿m ﻿“ ﻿6“BmW 6(“mWI“ ﻿6W6“ ﻿6“_(mHW “mWI“ ﻿m “ ﻿6“_(dB

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN